# Getting the data

In [1]:
import preprocessing
import featureSelection
import time

def eval_metrics(y, y_predicted):
    from sklearn.metrics import accuracy_score , precision_score , recall_score , f1_score
    print(accuracy_score(y, y_predicted))
    print(precision_score(y, y_predicted))
    print(recall_score(y, y_predicted))
    print(f1_score(y, y_predicted))

In [2]:
# corf = featureSelection.cor_feature[:23]
# print(corf)

# chif = featureSelection.chi_feature[:23]
# print(chif)
# # corfm = corf[:10] + corf[20:]
# # print(len(corfm))

features_extracted = ['Destination Port', 'Flow Duration', 'Bwd Packet Length Max', 'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow IAT Max', 'Fwd IAT Total', 'Fwd IAT Std', 'Fwd IAT Max', 'Max Packet Length', 'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance', 'PSH Flag Count', 'URG Flag Count', 'Avg Bwd Segment Size']


feature_list = features_extracted
print(len(feature_list))

16


In [3]:
X_train, y_train = featureSelection.X_train, featureSelection.y_train
X_train = X_train[feature_list]

In [4]:
X_cv, y_cv = featureSelection.X_cv, featureSelection.y_cv
X_cv = X_cv[feature_list]

In [5]:
X_train.info()
X_cv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2545088 entries, 0 to 2545087
Data columns (total 16 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   Destination Port        int64  
 1   Flow Duration           int64  
 2   Bwd Packet Length Max   int64  
 3   Bwd Packet Length Mean  float64
 4   Bwd Packet Length Std   float64
 5   Flow IAT Max            int64  
 6   Fwd IAT Total           int64  
 7   Fwd IAT Std             float64
 8   Fwd IAT Max             int64  
 9   Max Packet Length       int64  
 10  Packet Length Mean      float64
 11  Packet Length Std       float64
 12  Packet Length Variance  float64
 13  PSH Flag Count          int64  
 14  URG Flag Count          int64  
 15  Avg Bwd Segment Size    float64
dtypes: float64(7), int64(9)
memory usage: 310.7 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141394 entries, 0 to 141393
Data columns (total 16 columns):
 #   Column                  Non-Null Count   Dtype  
--- 

# Neural Network

In [ ]:
import tensorflow as tf
import tensorflow.keras
from numpy import loadtxt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, BatchNormalization, Dropout
import random
from numpy.random import seed
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.optimizers import Adam


# X_train = X_train.loc[:65535]
# y_train = y_train[:65536]
model = Sequential()
model.add(Dense(2*len(feature_list), input_dim=len(feature_list)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))
# model.add(Dense(2*len(feature_list), activation='relu'))
# model.add(Dense(len(feature_list), activation='relu'))
# model.add(Dense(len(feature_list), activation='relu'))
# model.add(Dense(len(feature_list), activation='relu'))
# model.add(Dense(len(feature_list), activation='relu'))
model.add(Dense(len(feature_list), activation='relu'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(len(feature_list)//2, activation='relu'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10, batch_size=256)


In [ ]:
model.save('nn.h5')

In [ ]:
print('Train metrics')
t_train = time.time()
y_train_pred = (model.predict(X_train) > 0.5).astype("int32")
print('Pred time on train set: ',time.time()-t_train)
eval_metrics(y_train, y_train_pred)
print('CV metrics')
t_cv = time.time()
y_cv_pred = (model.predict(X_cv) > 0.5).astype("int32")
print('Pred time on cv set: ',time.time()-t_cv)
eval_metrics(y_cv, y_cv_pred)

In [ ]:
X_test, y_test = featureSelection.get_x_y(featureSelection.test)
# print(X_test.info)
y_test_pred =(model.predict(X_test[feature_list]) > 0.5).astype("int32")
# print(y_test_pred)
eval_metrics(y_test, y_test_pred)

# Ensemble Learning

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score , precision_score , recall_score , f1_score

In [ ]:
rdf_c = RandomForestClassifier(n_estimators=300,random_state=0)
knn_clf = KNeighborsClassifier()
ExtraTrees_clf = ExtraTreesClassifier(n_estimators=100)
gradient_boost = GradientBoostingClassifier()
ada_mod = AdaBoostClassifier(base_estimator= None)
bg_mod = BaggingClassifier(base_estimator= None, bootstrap_features=False)

In [ ]:
vot_classifier = VotingClassifier(estimators=[
    ('bagging_classifier', bg_mod),
    ('rnd_classifier', rdf_c),
    ('ExtraTreesClassifier', ExtraTrees_clf),
    ('KNN_classifier', knn_clf),
],voting='soft', n_jobs=-1)

vot_classifier.fit(X_train, y_train)

In [ ]:
print('Train metrics')
t_train = time.time()
y_train_pred = vot_classifier.predict(X_train)
print('Pred time on train set: ',time.time()-t_train)
eval_metrics(y_train, y_train_pred)
print('CV metrics')
t_cv = time.time()
y_cv_pred = vot_classifier.predict(X_cv)
print('Pred time on cv set: ',time.time()-t_cv)
eval_metrics(y_cv, y_cv_pred)

In [ ]:
for clf in (ExtraTrees_clf, knn_clf, vot_classifier):
    clf.fit(X_train, y_train)
    y1 = clf.predict(X_train)
    print(clf.__class__.__name__, accuracy_score(y_train, y1))
    print(precision_score(y_train, y1))
    print(recall_score(y_train, y1))
    print(f1_score(y_train, y1))

In [ ]:
for clf in (ExtraTrees_clf, knn_clf, vot_classifier):
    clf.fit(X_cv, y_cv)
    y1 = clf.predict(X_cv)
    print(clf.__class__.__name__, accuracy_score(y_cv, y1))
    print(precision_score(y_cv, y1))
    print(recall_score(y_cv, y1))
    print(f1_score(y_cv, y1))

# ExtraTrees

In [15]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import make_pipeline

mm_scaler = MinMaxScaler()
ExtraTrees_clf = ExtraTreesClassifier(n_estimators=50)

pipeline = make_pipeline(mm_scaler, ExtraTrees_clf)
pipeline.fit(X_train, y_train)

Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                ('extratreesclassifier',
                 ExtraTreesClassifier(n_estimators=50))])

In [16]:
print('Train metrics')
t_train = time.time()
y_train_pred = pipeline.predict(X_train)
print('Pred time on train set: ',time.time()-t_train)
eval_metrics(y_train, y_train_pred)
print('CV metrics')
t_cv = time.time()
y_cv_pred = pipeline.predict(X_cv)
print('Pred time on cv set: ',time.time()-t_cv)
eval_metrics(y_cv, y_cv_pred)

Train metrics
Pred time on train set:  21.26222538948059
0.9983914112203586
0.9941868099074903
0.9976602116190857
0.995920482287878
CV metrics
Pred time on cv set:  1.2038090229034424
0.9974609955160757
0.9916944116278237
0.9954362512577261
0.993561808432417


In [ ]:
X_test, y_test = featureSelection.get_x_y(featureSelection.test)
y_test_pred = pipeline.predict(X_test[feature_list])
eval_metrics(y_test, y_test_pred)

In [36]:
import joblib 
# joblib.dump(pipeline, 'extratrees.sav')
model = joblib.load('extratrees.sav')

0.9974609955160757
0.9916944116278237
0.9954362512577261
0.993561808432417
0.9976802410286151
0.9926549623790756
0.9955799913755929
0.9941153252718075
